### Pipeline test ###

In this notebook we will try a possible textual claim generation to be implemented later as part of the package

In [ ]:
from hydra.core.global_hydra import GlobalHydra
from hydra import initialize,compose
from src.evidence.feverous_retriever.random import FeverousRetrieverRandom
from src.pipeline import ClaimGeneratorPipeline
from src.claim import FeverousGenerator
from src.claim import ToTToGenerator
from src.claim import TextualClaim
import numpy as np
import json
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
GlobalHydra.instance().clear()
initialize(config_path='../src/config/', job_name="test_app")

In [ ]:
cfg = compose(config_name="config_pipeline")
rng = np.random.default_rng(cfg.seed)
retrievers = [FeverousRetrieverRandom(p_dataset=cfg.notebook.data_path,
                                    num_positive=10,
                                    num_negative=0,
                                    wrong_cell=2,
                                    table_per_page=cfg.table_per_page,
                                    evidence_per_table=cfg.evidence_per_table,
                                    column_per_table=cfg.column_per_table,
                                    seed=44,
                                    key_strategy=strat,
                                    verbose=True
                                    ) for strat in ['entity']]


generator1 = FeverousGenerator(encoding='compact', model_path='../models/t5_train_feverous.bin')
#generator2 = ToTToGenerator(encoding='totto', model_path='../models/exported_totto_large/1648208035')
generator3 = ToTToGenerator(encoding='compact', model_path='../models/exported_totto_large/1648208035', verbose=True)

generators = [generator3]

pipeline = ClaimGeneratorPipeline([retrievers,generators])
claims = pipeline.generate() # Right now, FeverousRetriever doesn't support an input table


json_evidence = TextualClaim.to_json(claims)
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(json_evidence, f, ensure_ascii=False, indent=4)

